In [76]:
D_path='Data_1001_05\\'
ID='05931292'
O_path='CSV\\'
headerTF=False

In [132]:
n=0
for file in os.listdir(D_path):
    p=re.compile('(\d{8})_culture.txt')
    res=p.match(file)
    if res:
#         print(res[1])
        Write_CSV(res[1],D_path,O_path,False)
        if not Check_Organism(res[1],D_path):
            print(res[1])
        n=n+1
    else:
        print(file)
print(n)

815


In [40]:
p=re.compile('(\d{8})_culture.txt')
res=p.match('00455289_culture.txt')
res[1]

'00455289'

In [125]:
import pandas as pd
import re
import os


def Get_Record(ID,path=''):
    index=0
    result=list()

    with open(path+ID+"_culture.txt", "r",encoding='utf-8') as f:
        while(True):
            line=f.readline()
            if line=='=================================================================================\n':
                if index!=0:
                    result=result+[[ID,lab_code,lab_name,lab_performed_time,lab_reported_time,specimen,comment,Cx_result,sp1,sp2,sp3,sp4]]
                index=index+1
#                 print("시작")
                
                while(True): #공백 
                    line=f.readline()
                    if line !='\n':
                        break
#                 print(line)
                #검사명
#                 line=f.readline()
                p=re.compile('^【검사명】([^/[]+)\\[(\w+)\\]')
                res=p.match(line)
                lab_name=res.group(1).strip()
                lab_code=res.group(2) 

    #             print(line[:-1])
    #             print(lab_name,lab_code)

                while(True): #공백
                    line=f.readline()
                    if line !='\n':
                        break
#                 f.readline() #공백
#                 print(line)

                #시각
#                 line=f.readline()
                p=re.compile('【검사일】(\d\d\d\d-\d\d-\d\d \d\d:\d\d) 【보고일】(\d\d\d\d-\d\d-\d\d \d\d:\d\d)')
                res=p.search(line)
                lab_performed_time=res.group(1)
                lab_reported_time=res.group(2)
    #             print(line[:-1])
    #             print(lab_performed_time,lab_reported_time)

                while(True): #공백
                    line=f.readline()
                    if line=='=================================================================================\n':
                        break
#                 f.readline() #공백
#                 f.readline() #라인
                Cx_result=False
                specimen=''
                sp1=sp2=sp3=sp4=''
                comment=''

            elif not line:
                if index!=0:
                    result=result+[[ID,lab_code,lab_name,lab_performed_time,lab_reported_time,specimen,comment,Cx_result,sp1,sp2,sp3,sp4]]
    #             print("종료")
                break
            p=re.compile(' 【검체】 (.+)')
            res=p.match(line)
            if res:
                specimen=res.group(1)

            p=re.compile('임상소견 : (.+)')
            res=p.search(line)
            if res:
                comment=res.group(1)
                
            p=re.compile('Isolate  : #01')
            res=p.search(line)
            if res:
                Cx_result=True
                p=re.compile('Organism : (.+)')
                while(True): #공백
                    line=f.readline()
                    res=p.search(line)
                    if res:
                        sp1=res.group(1)
                        break

            p=re.compile('Isolate  : #02')
            res=p.search(line)
            if res:
                Cx_result=True
                p=re.compile('Organism : (.+)')
                while(True): #공백
                    line=f.readline()
                    res=p.search(line)
                    if res:
                        sp2=res.group(1)
                        break

            p=re.compile('Isolate  : #03')
            res=p.search(line)
            if res:
                Cx_result=True
                p=re.compile('Organism : (.+)')
                while(True): #공백
                    line=f.readline()
                    res=p.search(line)
                    if res:
                        sp3=res.group(1)
                        break
                        
            p=re.compile('Isolate  : #04')
            res=p.search(line)
            if res:
#                 print(ID)
                Cx_result=True
                p=re.compile('Organism : (.+)')
                while(True): #공백
                    line=f.readline()
                    res=p.search(line)
                    if res:
                        sp4=res.group(1)
                        break                        

    return (pd.DataFrame(result,columns =['ID','lab_code','lab_name','lab_performed_time','lab_reported_time','specimen','comment','Cx_result','sp1','sp2','sp3','sp4']))
# print(index)
# result

def Write_CSV(ID,D_path,O_path,headerTF=True):
    df=Get_Record(ID,path=D_path)
    df.to_csv(O_path+ID+'_csv.txt',index=False,header=headerTF)

In [172]:
def Check_Organism(ID,path=''):
    index=0
    result=list()
    last_line=''

    with open(path+ID+"_culture.txt", "r",encoding='utf-8') as f:
        while(True):
            line=f.readline()
            if not line:
                break
            
            
            p=re.compile('Organism : (.+)')
            res=p.search(line)
            if res:
                p=re.compile('Isolate  : ')
                if not p.search(last_line):
#                     print(last_line)
#                     print(line)
                    return False
            if line!='\n':
                last_line=line

    return True

def Check_Name(name):
    p=re.compile('#\d$')
    if not p.search(name):
        return(True)
    return(name.endswith('#1'))

def str2date(dstring):
    return (datetime.strptime(dstring.strip(),'%Y-%m-%d'))

In [153]:
import pandas as pd
from datetime import datetime

In [142]:
raw_cx=pd.read_csv('culture_merge.txt',dtype={'ID': str},encoding='utf-8')
ECMO_list=pd.read_csv('list_csv.txt',dtype={'ID': str,'Duration':int})

In [175]:
raw_cx
# ECMO_list

,ID,lab_code,lab_name,lab_performed_time,lab_reported_time,specimen,comment,Cx_result,sp1,sp2,sp3,sp4
0,00455289,BL4001,"Blood Culture, 1st",2019-04-09 08:44,2019-04-14 08:53,"Blood, Central line",NaN,False,NaN,NaN,NaN,NaN
1,00455289,BL4001,"Blood Culture, 1st",2019-04-09 08:44,2019-04-14 08:52,"Blood,Peripheral",NaN,False,NaN,NaN,NaN,NaN
2,00455289,BL4001,"Blood Culture, 1st",2019-04-12 10:46,2019-04-17 10:50,"Blood,Peripheral",NaN,False,NaN,NaN,NaN,NaN
3,00455289,BL4001,"Blood Culture, 1st",2019-04-12 10:46,2019-04-17 10:48,"Blood, Central line",NaN,False,NaN,NaN,NaN,NaN
4,00455289,BL4001,"Blood Culture, 1st",2019-04-15 10:59,2019-04-20 11:09,"Blood, Central line",NaN,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
17890,44714654,BL4020,"Gram Stain and Culture, Bacteria",2020-07-07 20:27,2020-07-10 10:04,"Urine, Foley cath",NaN,False,NaN,NaN,NaN,NaN
17891,44714654,BL4020,"Gram Stain and Culture, Bacteria",2020-07-07 20:27,2020-07-10 09:34,(Endo)Tracheal Aspir,NaN,False,NaN,NaN,NaN,NaN
17892,44714654,BL4020,"Gram Stain and Culture, Bacteria",2020-07-08 15:09,2020-07-10 09:57,혈관내 카테터,NaN,False,NaN,NaN,NaN,NaN
17893,44714654,BL4020,"Gram Stain and Culture, Bacteria",2020-07-09 10:49,2020-07-11 10:56,(Endo)Tracheal Aspir,NaN,False,NaN,NaN,NaN,NaN


In [191]:
raw_cx=pd.read_csv('culture_merge.txt',dtype={'ID': str},encoding='utf-8')
ECMO_list=pd.read_csv('list_csv.txt',dtype={'ID': str,'Duration':int})

err_list=list()

for i in range(len(ECMO_list)):
    if i==180 or i==189:
        continue
    if Check_Name(ECMO_list['이름'][i]):
#         print(i)
        start=str2date(ECMO_list['Lab 시작'][i][:10])
        end=str2date(ECMO_list['ECMO 제거일'][i][:10])
        
        for j in raw_cx[raw_cx.ID==ECMO_list['ID'][i]].lab_performed_time:
            lab=str2date(j[:10])
            if lab<start or lab>end:
                print(ECMO_list['ID'][i],start,end,lab)
                err_list=err_list+[[ECMO_list['ID'][i],start,end,lab]]
       
       
err_list

[]

In [187]:
i=1
for j in raw_cx[raw_cx.ID==ECMO_list['ID'][i]].lab_performed_time:
    print(str2date(j[:10]))


2013-12-18 00:00:00
2014-01-03 00:00:00
2014-01-03 00:00:00
2013-12-18 00:00:00
2013-12-18 00:00:00
2013-12-10 00:00:00
2013-12-11 00:00:00
2013-12-11 00:00:00
2013-12-11 00:00:00
2013-12-11 00:00:00
2014-01-03 00:00:00
2014-01-04 00:00:00


In [ ]:
from datetime import datetime

date_time_str = '18/09/19 01:55:19'

date_time_obj = datetime.strptime(date_time_str, '%d/%m/%y %H:%M:%S')


print "The type of the date is now",  type(date_time_obj)
print "The date is", date_time_obj